In [1]:
import pandas as pd
import numpy as np
import time
import gc

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

# Get Train on Day7 Day8 Day9

In [5]:
df_hour = pd.read_csv(path+'hourdistri.csv', index_col='Unnamed: 0')
index = {}
for day in ['day7', 'day8','day9']:
    index[day] = list(range(df_hour.loc[day,'4start'], df_hour.loc[day,'6end0sec'])) + \
    list(range(df_hour.loc[day,'9start'], df_hour.loc[day,'11end0sec'])) + \
    list(range(df_hour.loc[day,'13start'], df_hour.loc[day,'15end0sec'])) 

In [6]:
history_index = list(set(train.index) - set(index['day7']) - set(index['day8']) - set(index['day9']))

In [9]:
for file in ['day7', 'day8', 'day9','test']: 
    print(len(df_dict[file]))

19534560
20446743
20898422
18790469


# Define functin

In [8]:



def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  

    group = get_group(df_train, cols)
    group_history = get_group(df_history, cols)
    mean_map = df_history.groupby(group_history)[target].mean()
    return group.map(mean_map).fillna(-0.01)


def discre_encode(df_all, df, test,cols, target, scale=10):
    group = get_group(df, cols)
    group_test = get_group(test, cols)
    group_all = get_group(df_all, cols)
    mean_map = df_all.groupby(group_all)[target].mean()
    # old 
#     mean_map = (mean_map * scale)
    #### updated
    mean_map_zero = mean_map[mean_map == 0]
    mean_map_p = mean_map[mean_map > 0]
    mean_map_p = (mean_map_p * scale) + 1
    mean_map = pd.concat([mean_map_zero, mean_map_p])
    #end
    intersection = list(set(group_all.value_counts().index).intersection(set(group_test.value_counts().index)))
    mean_map = mean_map.loc[intersection]
    return group.map(mean_map).fillna(-1).astype('int32')




def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__
    
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',]

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(train[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group


# Feature Engineering

In [9]:
scale = 10
target='is_attributed'
for day in ['day7', 'day8','day9']:
    print('processing {}'.format(day))
    df_day = pd.DataFrame()
    save_file_name = '{}_category_discrete_scale{}_newhistory.csv'.format(day,scale)
    for col in ['ip', 'channel', 'device', 'app', 'os']:
        print('processing {}'.format(col))
        df_day[col] = discre_encode(train.iloc[history_index], train.iloc[index[day]],test, [col], target, scale)
    save_file_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/' + save_file_name
    df_day.to_csv(save_file_path, index=False)
    print(len(df_day))
    print('==================================================')

processing day7
processing ip
processing channel
processing device
processing app
processing os
19534560
processing day8
processing ip
processing channel
processing device
processing app
processing os
20446743
processing day9
processing ip
processing channel
processing device
processing app
processing os
20898422


# Feature engineering on Test

In [10]:
scale = 10

print('processing {}'.format('test'))
df_day = pd.DataFrame()
save_file_name = '{}_category_discrete_scale{}_newhistory.csv'.format('test',scale)

for col in ['ip', 'channel', 'device', 'app', 'os']:
    print('processing {}'.format(col))
    df_day[col] = discre_encode(train, test,test, [col], target, scale)
save_file_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/' + save_file_name
df_day.to_csv(save_file_path, index=False)
print(len(df_day))
print('==================================================')

processing test
processing ip
processing channel
processing device
processing app
processing os
18790469


In [4]:
## load category col
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_category_discrete_scale10_new.csv'
df_dict_cat = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict_cat[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_category_discrete_scale10_new.csv
day8_category_discrete_scale10_new.csv
day9_category_discrete_scale10_new.csv
test_category_discrete_scale10_new.csv


In [6]:
df_dict_cat['test'].ip.value_counts()

 1     16704143
-1      1527103
 0       553812
 2         1945
 3         1149
 11         768
 4          639
 6          580
 5          143
 7           76
 10          46
 8           37
 9           28
Name: ip, dtype: int64

In [11]:
for file in ['day7', 'day8', 'day9','test']: 
    print(len(df_dict_cat[file]))

19534560
20446743
20898422
18790469
